In [138]:
# Dependencies
import pandas as pd
from pathlib import Path

In [139]:
# To load files
school_complete_load = Path("Resources\schools_complete.csv")
student_complete_load = Path("Resources\students_complete.csv")

# To read files 
school_data = pd.read_csv(school_complete_load)
student_data = pd.read_csv(student_complete_load)

In [140]:
# Merge 
school_data_merged = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_merged

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12,Thomas High School,51,48,14,Independent,1635,1043130
39166,39166,Dawn Bell,F,10,Thomas High School,81,89,14,Independent,1635,1043130
39167,39167,Rebecca Tanner,F,9,Thomas High School,99,99,14,Independent,1635,1043130
39168,39168,Desiree Kidd,F,10,Thomas High School,72,77,14,Independent,1635,1043130


In [141]:
# Number of unique schools
total_unique_schools = school_data_merged['school_name'].nunique()

# Number of students 
total_students = len(school_data_merged)

# Budget 
total_budget = school_data['budget'].sum()

# Mean maths score 
mean_maths_score = school_data_merged['maths_score'].mean()

# Mean reading scores 
mean_reading_score = school_data_merged['reading_score'].mean()

# Percentage students who passed maths 
students_passed_maths = school_data_merged[school_data_merged['maths_score']>=50]
percentage_passed_math = (len(students_passed_maths)/total_students) * 100 

# Percentage students who passed reading 
students_passed_reading = school_data_merged[school_data_merged['reading_score']>=50]
percentage_passed_reading = (len(students_passed_reading)/total_students) * 100

# Percentage of overall passing (both maths and reading)
students_passed_both = school_data_merged[(school_data_merged['maths_score']>=50) & (school_data_merged['reading_score']>=50)]
percentage_passed_both = (len(students_passed_both)/total_students) * 100 

LGA_summary = pd.DataFrame({
    'Total Schools': '{:,}'.format(total_unique_schools),
    'Total Students': [total_students],
    'Total Budget': ['${:,.2f}'.format(total_budget)],
    'Average Maths Score': ['{:,.2f}%'.format(mean_maths_score)],
    'Average Reading Score': ['{:,.2f}%'.format(mean_reading_score)],
    '% Passing Maths': ['{:,.2f}%'.format(percentage_passed_math)],
    '% Passing Reading': ['{:,.2f}%'.format(percentage_passed_reading)],
    '% Overall Passing': ['{:,.2f}%'.format(percentage_passed_both)]
})

LGA_summary


,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",70.34%,69.98%,86.08%,84.43%,72.81%


In [142]:
# School Summary 

# School types 
school_types = school_data.set_index(['school_name'])["type"]

# Calculate total student per school 
per_school_count = school_data_merged['school_name'].value_counts()

# Calculate total school budget and spending per school
per_school_budget = school_data_merged.groupby(['school_name'])['budget'].mean()
per_school_capita = per_school_budget / per_school_count 


In [143]:
#Calculate average test scores by school 
per_school_maths = school_data_merged.groupby(['school_name'])['maths_score'].mean()
per_school_reading = school_data_merged.groupby(['school_name'])['reading_score'].mean()

In [144]:
# Students who passed maths and passed reading  
passing_maths = school_data_merged[(school_data_merged['maths_score']>= 50)]
passing_reading = school_data_merged[(school_data_merged['reading_score']>= 50)]

# Stud.coun...gbwdesb bhbbb!. dsb ents who passed both 
passing_maths_reading = school_data_merged[(school_data_merged['maths_score']>=50) & (school_data_merged['reading_score']>=50)]

In [145]:
# Calculate % Pass
per_school_passing_maths = passing_maths.groupby(['school_name']).size() / per_school_count * 100 
per_school_passing_reading = passing_reading.groupby(['school_name']).size() / per_school_count * 100 
overall_passing_rate = passing_maths_reading.groupby(['school_name']).size() / per_school_count * 100

In [146]:
# Reset the index to make the school_name a separate column
#school_data_merged.reset_index(inplace=True)

# Display the School Summary DataFrame
school_summary = pd.DataFrame({
    'School Type': school_types,
    'Total Students': per_school_count,
    'Total School Budget': per_school_budget,
    'Per Student Budget': per_school_capita,
    'Average Maths Score': per_school_maths,
    'Average Reading Score': per_school_reading, 
    '% Passing Maths': per_school_passing_maths,
    '% Passing Reading': per_school_passing_reading,
    '% Overall Passing': overall_passing_rate
})

school_summary.rename(columns={'school_name': 'School Name'}, inplace=True)
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Bailey High School,Government,4976,3124928.0,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,1081356.0,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,1884411.0,639.0,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,1763916.0,644.0,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,917500.0,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,3022020.0,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,248087.0,581.0,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,1910635.0,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,3094650.0,650.0,68.843100,69.039277,82.062592,81.978576,67.191766
Pena High School,Independent,962,585858.0,609.0,72.088358,71.613306,91.683992,86.590437,79.209979


In [147]:
# Highest-Performing Schools (by % Overall Passing)
top_schools = school_summary.sort_values(by= '% Overall Passing', ascending=False).head(5)
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Griffin High School,Independent,1468,917500.0,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,1081356.0,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,3124928.0,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,1049400.0,583.0,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,2547363.0,637.0,72.047762,70.935984,90.797699,87.396849,79.419855


In [148]:
# Lowest-Performing Schools (by % Overall Passing)
bottom_schools = school_summary.sort_values(by='% Overall Passing', ascending=True).head(5)
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Hernandez High School,Government,4635,3022020.0,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,1910635.0,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,3094650.0,650.0,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,1319574.0,578.0,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,1763916.0,644.0,69.091274,69.572472,82.438846,82.219788,67.469880


In [149]:
# Maths Scores by Year
average_math_score_by_year = school_data_merged.groupby(['school_name', 'year'])['maths_score'].mean().reset_index()
pivot_table = average_math_score_by_year.pivot(index='school_name', columns='year', values='maths_score')
pivot_table.columns = ['Year ' + str(year) for year in pivot_table.columns]
pivot_table.index.name = None
pivot_table

,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


In [150]:
# Reading Scores by Year
average_reading_score_by_year = school_data_merged.groupby(['school_name', 'year'])['reading_score'].mean().reset_index()
pivot_table = average_reading_score_by_year.pivot(index='school_name', columns='year', values='reading_score')
pivot_table.columns = ['Year ' + str(year) for year in pivot_table.columns]
pivot_table.index.name = None
pivot_table

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


In [84]:
# Scores by School Spending 


spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

school_summary['Spending Ranges (Per Student)'] = pd.cut(school_summary['Spending Ranges (Per Student)'], bins=spending_bins, labels=labels, right=False)

# Mean scores per spending range 


KeyError: 'Spending Ranges (Per Student)'

In [ ]:
# Scores by School Size 

size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary['']